CURATION SCRIPT
by: Igor Sanches

**IMPORT DEPENDENCIES AND IN-HOUSE FUNCTIONS**

In [ ]:
#import libraries and dependencies
import pandas as pd
import math
import numpy as np

from rdkit import Chem
from chembl_structure_pipeline import standardizer
from rdkit.Chem.MolStandardize.metal import MetalDisconnector
import rdkit.Chem.MolStandardize.rdMolStandardize as rdMolStandardize
from rdkit.Chem import PandasTools

from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import inchi as rd_inchi

from molvs import standardize_smiles
from molvs import Standardizer
from rdkit.Chem import Draw

In [ ]:
#in-house functions
def metal_atomic_numbers(at):
    """ This function checks the atomic number of an atom """
    
    n = at.GetAtomicNum()
    return (n==13) or (n>=21 and n<=31) or (n>=39 and n<=50) or (n>=57 and n<=83) or (n>=89 and n<=115)

def is_metal(smile):
    """ This function checks if an atom is a metal based on its atomic number """
    mol = Chem.MolFromSmiles(smile)
    rwmol = Chem.RWMol(mol)
    rwmol.UpdatePropertyCache(strict=False)
    metal = [at.GetSymbol() for at in rwmol.GetAtoms() if metal_atomic_numbers(at)]
    return len(metal) == 1

def smiles_preparator(smile):
    """ This function prepares smiles by removing stereochemistry """
    smile1 = smile.replace('@','')
    smile2 = smile1.replace('/','')
    smile3 = smile2.replace("\\",'')
    return str(smile3)

def neutralizeRadicals(mol):
    for a in mol.GetAtoms():
        if a.GetNumRadicalElectrons()==1 and a.GetFormalCharge()==1:
            a.SetNumRadicalElectrons(0)         
            a.SetFormalCharge(0)
            
def salt_remover(mol):
    """ This function removes salts, see complete list of possible salts in https://github.com/rdkit/rdkit/blob/master/Data/Salts.txt """
    remover1 = SaltRemover(defnData=None)
    remover2 = SaltRemover(defnData="[Cl,Br,I]")
    remover3 = SaltRemover(defnData="[Li,Na,K,Ca,Mg]")
    remover4 = SaltRemover(defnData="[O,N]")
    remover5 = SaltRemover(defnData="[H]")
    remover6 = SaltRemover(defnData="[Ba]")
    remover7 = SaltRemover(defnData="[Al]")
    remover8 = SaltRemover(defnData="[Cu]")
    remover9 = SaltRemover(defnData="[Cs]")
    remover10 = SaltRemover(defnData="[Zn]")
    remover11 = SaltRemover(defnData="[Mn]")
    remover12 = SaltRemover(defnData="Cl[Cr]Cl")
    remover13 = SaltRemover(defnData="COS(=O)(=O)[O-]")
    remover14 = SaltRemover(defnData="[Sb]")
    remover15 = SaltRemover(defnData="[Cr]")
    remover16 = SaltRemover(defnData="[Ni]")
    remover17 = SaltRemover(defnData="[B]")
    remover18 = SaltRemover(defnData="CCN(CC)CC")
    remover19 = SaltRemover(defnData="NCCO")
    remover20 = SaltRemover(defnData="O=CO")
    remover21 = SaltRemover(defnData="O=S(=O)([O-])C(F)(F)F")
    stripped1 = remover1.StripMol(mol, dontRemoveEverything=True)
    stripped2 = remover2.StripMol(stripped1, dontRemoveEverything=True)
    stripped3 = remover3.StripMol(stripped2, dontRemoveEverything=True)
    stripped4 = remover4.StripMol(stripped3, dontRemoveEverything=True)
    stripped5 = remover5.StripMol(stripped4, dontRemoveEverything=True)
    stripped6 = remover6.StripMol(stripped5, dontRemoveEverything=True)
    stripped7 = remover7.StripMol(stripped6, dontRemoveEverything=True)
    stripped8 = remover8.StripMol(stripped7, dontRemoveEverything=True)
    stripped9 = remover9.StripMol(stripped8, dontRemoveEverything=True)
    stripped10 = remover10.StripMol(stripped9, dontRemoveEverything=True)
    stripped11 = remover11.StripMol(stripped10, dontRemoveEverything=True)
    stripped12 = remover12.StripMol(stripped11, dontRemoveEverything=True)
    stripped13 = remover13.StripMol(stripped12, dontRemoveEverything=True)
    stripped14 = remover14.StripMol(stripped13, dontRemoveEverything=True)
    stripped15 = remover15.StripMol(stripped14, dontRemoveEverything=True)
    stripped16 = remover16.StripMol(stripped15, dontRemoveEverything=True)
    stripped17 = remover17.StripMol(stripped16, dontRemoveEverything=True)
    stripped18 = remover18.StripMol(stripped17, dontRemoveEverything=True)
    stripped19 = remover19.StripMol(stripped18, dontRemoveEverything=True)
    stripped20 = remover20.StripMol(stripped19, dontRemoveEverything=True)
    stripped21 = remover21.StripMol(stripped20, dontRemoveEverything=True)
    return stripped21

**SET PATH**

In [ ]:
#choose a path to save
savepath = r'D:\python\jupyterscripts\curation'

In [ ]:
#import dataset
df4 = pd.read_csv(r'D:\python\jupyterscripts\curation\natura\data_natura.csv', encoding='latin-1')

df4

**DATA PREPARATION**

In [ ]:
#remove unwanted columns
dropList = ['Standard Type', 'Standard Units', 'Assay ChEMBL ID', 'Assay Description', 'Document ChEMBL ID', 'Assay Cell Type']
df1 = df0.drop(columns = dropList)
df1

In [ ]:
#rename columns
df1.rename(columns = {'Molecule ChEMBL ID':'ID', 'Smiles':'SMILES', 'Standard Value':'IC50 (nM)', 'Standard Relation':'Relation'}, inplace = True)

In [ ]:
#check for unique values before deleting
#df1['Relation'].unique()

In [ ]:
#drop rows with missing values (only drop rows with missing values on activity column)
df4 = df1.dropna(subset=['IC50 (nM)'])

#total removed with activity missing values 
total_removed1 = len(df1)-len(df4)

#drop rows with missing values (only drop rows with missing values on relation column)
#df3 = df2.dropna(subset=['Relation'])

#total removed with relation missing values 
#total_removed2 = len(df2)-len(df3)
total_removed1

In [ ]:
#drop all values != '='
#df4 = df3[df3.Relation == "'='"]

#total removed with relation =! '=' 
#total_removed3 = len(df3)-len(df4)

#check for unique values before deleting
#df4['Relation'].unique()

In [ ]:
ic50uuM = []
for value in df4['IC50 (nM)']:
    ic50uM = value / 1000
    ic50uuM.append(ic50uM)
df4['IC50 (uM)'] = ic50uuM

In [ ]:
df4

In [ ]:
#convert to pIC50
pic50 = []
for value in df4['IC50 (uM)']:
    pic50.append(-(math.log10(value*10**-6)))

df4['pIC50 (uM)'] = pic50

In [ ]:
df4

In [ ]:
#remove stereoisomers 
smiles = [smiles_preparator(str(smile)) for smile in df4['SMILES']]
df4['SMILES_no_stereo'] = smiles

#remove relation column
#df4 = df4.drop(columns = 'Relation')
df4

In [ ]:
df4 = df4.drop(columns = 'IC50 (nM)')
df4

**REMOVE SALTS AND INVALID SMILES**

In [ ]:
#remove salts
wrongSmiles = []
new_smiles = []
indexDropList_salts = []
for index, smile in enumerate(df4['SMILES_no_stereo']):
    try:
        mol = Chem.MolFromSmiles(smile)
        remov = salt_remover(mol)
        if remov.GetNumAtoms() <= 2:
            indexDropList_salts.append(index)
        else:
            new_smiles.append(Chem.MolToSmiles(remov, kekuleSmiles=False))
        
    except:
        wrongSmiles.append(df4.iloc[[index]])
        indexDropList_salts.append(index)


if len(wrongSmiles) == 0:
    print("no wrong smiles found")
    
else:
    #drop wrong smiles
    df4 = df4.drop(df4.index[indexDropList_salts])
    
    print(f"{len(wrongSmiles)} wrong smiles found")
    
    #save removes mixtures
    wrongsmiles = pd.concat(wrongSmiles)
    wrongsmiles.to_csv(f'{savepath}\\wrongsmiles.csv', sep=',', header=True, index=False)
df4['SMILES_no_salts'] = new_smiles
df4 

**REMOVE ORGANOMETALLICS**

In [ ]:
organometals = []
indexDropList_org = []
for index, smile in enumerate(df4['SMILES_no_salts']):
    if is_metal(smile) == True:
        organometals.append(df4.iloc[[index]])
        indexDropList_org.append(index)

if len(indexDropList_org) == 0:
    print("no organometallics found")
    
else:
    #drop organometallics
    df4 = df4.drop(df4.index[indexDropList_org])
    
    print(f"{len(indexDropList_org)} organometallics found")
    
    #save droped organometallics
    organmetal = pd.concat(organometals)
    organmetal.to_csv(f'{savepath}\\organometallics.csv', sep=',', header=True, index=False)
df4

**REMOVE MIXTURES**

In [ ]:
#remove mixtures
mixtureList = []
indexDropList_mix = []
for index, smile in enumerate (df4['SMILES_no_salts']):
    for char in smile:
        if char == '.':
            mixtureList.append(df4.iloc[[index]])
            indexDropList_mix.append(index)
            break

            
if len(indexDropList_mix) == 0:
    print("no mixtures found")
    
else:
    #drop mixtures
    df4 = df4.drop(df4.index[indexDropList_mix])
    
    print(f"{len(indexDropList_mix)} mixtures found")
    
    #save removes mixtures
    mixtures = pd.concat(mixtureList)
    mixtures.to_csv(f'{savepath}\\mixtures.csv', sep=',', header=True, index=False)
df4 

**STANDARDISE**

In [ ]:
"""
    -Standardize unknown stereochemistry (Handled by the RDKit Mol file parser)
        Fix wiggly bonds on sp3 carbons - sets atoms and bonds marked as unknown stereo to no stereo
        Fix wiggly bonds on double bonds – set double bond to crossed bond
    -Clears S Group data from the mol file
    -Kekulize the structure
    -Remove H atoms (See the page on explicit Hs for more details)
    -Normalization:
        Fix hypervalent nitro groups
        Fix KO to K+ O- and NaO to Na+ O- (Also add Li+ to this)
        Correct amides with N=COH
        Standardise sulphoxides to charge separated form
        Standardize diazonium N (atom :2 here: [*:1]-[N;X2:2]#[N;X1:3]>>[*:1]) to N+
        Ensure quaternary N is charged
        Ensure trivalent O ([*:1]=[O;X2;v3;+0:2]-[#6:3]) is charged
        Ensure trivalent S ([O:1]=[S;D2;+0:2]-[#6:3]) is charged
        Ensure halogen with no neighbors ([F,Cl,Br,I;X0;+0:1]) is charged
    -The molecule is neutralized, if possible. See the page on neutralization rules for more details.
    -Remove stereo from tartrate to simplify salt matching
    -Normalise (straighten) triple bonds and allenes
    
    https://github.com/chembl/ChEMBL_Structure_Pipeline
"""

rdMol = [Chem.MolFromSmiles(smile, sanitize=True) for smile in df4['SMILES_no_salts']]

molBlock = [Chem.MolToMolBlock(mol) for mol in rdMol]

stdMolBlock = [standardizer.standardize_molblock(mol_block) for mol_block in molBlock]

molFromMolBlock = [Chem.MolFromMolBlock(std_molblock) for std_molblock in stdMolBlock]

mol2smiles = [Chem.MolToSmiles(m) for m in molFromMolBlock]

df4['Stand_smiles'] = mol2smiles

**Remove Salts for the second time**

In [ ]:
#remove salts second time
wrongSmiles = []
new_smiles = []
indexDropList_salts = []
for index, smile in enumerate(df4['Stand_smiles']):
    try:
        mol = Chem.MolFromSmiles(smile)
        remov = salt_remover(mol)
        if remov.GetNumAtoms() <= 2:
            indexDropList_salts.append(index)
        else:
            new_smiles.append(Chem.MolToSmiles(remov, kekuleSmiles=False))
        
    except:
        wrongSmiles.append(df4.iloc[[index]])
        indexDropList_salts.append(index)


if len(wrongSmiles) == 0:
    print("no wrong smiles found")
    
else:
    #drop wrong smiles
    df4 = df4.drop(df4.index[indexDropList_salts])
    
    print(f"{len(indexDropList_salts)} wrong smiles found")
    
    #save removes mixtures
    wrongsmiles = pd.concat(wrongSmiles)
    wrongsmiles.to_csv(f'{savepath}\\wrongsmiles_after_std_2.csv', sep=',', header=True, index=False)
df4['secondSaltRemoved_smiles'] = new_smiles
df4 

**remove radicals and standalone salts**

In [ ]:
#remove radicals and standalone salts
mols_noradical = []
standAlone_salts = []
indexDropList_salts = []
for index, smile in enumerate(df4['secondSaltRemoved_smiles']):
    try:
        m = Chem.MolFromSmiles(smile, False)
        m = rd_inchi.MolToInchi(m)
        m = Chem.MolFromInchi(m)
        neutralizeRadicals(m)
        Chem.SanitizeMol(m)
        mols_noradical.append(Chem.MolToSmiles(m, False))
    except:
        indexDropList_salts.append(index)
        standAlone_salts.append(df4.iloc[[index]])
if len(standAlone_salts) == 0:
    print("No salts found")
else:
    df4 = df4.drop(df4.index[indexDropList_salts])
    salts = pd.concat(standAlone_salts)
    salts.to_csv(f'{savepath}\\removed_salts.csv', sep=',', header=True, index=False)
df4['removed_radicals_smile'] = mols_noradical

In [ ]:
#remove salts second time

new_smiles = []
indexDropList_salts = []
for index, smile in enumerate(df4['removed_radicals_smile']):
    try:
        mol = Chem.MolFromSmiles(smile)
        remov = salt_remover(mol)
        if remov.GetNumAtoms() <= 2:
            indexDropList_salts.append(index)
        else:
            new_smiles.append(Chem.MolToSmiles(remov, kekuleSmiles=False))
        
    except:
        wrongSmiles.append(df4.iloc[[index]])
        indexDropList_salts.append(index)


if len(indexDropList_salts) == 0:
    print("no wrong smiles found")
    
else:
    #drop wrong smiles
    df4 = df4.drop(df4.index[indexDropList_salts])
    
    print(f"{len(indexDropList_salts)} wrong smiles found")
    
    #save removes mixtures
    indexDropList_salts = pd.concat(indexDropList_salts)
    indexDropList_salts.to_csv(f'{savepath}\\wrongsmiles_after_std_3.csv', sep=',', header=True, index=False)
df4['thirdSaltRemoved_smiles'] = new_smiles
df4 

**Remove mixtures for the second time**

In [ ]:
#remove mixtures
mixtureList = []
indexDropList_mix = []
for index, smile in enumerate (df4['thirdSaltRemoved_smiles']):
    for char in smile:
        if char == '.':
            mixtureList.append(df4.iloc[[index]])
            indexDropList_mix.append(index)
            break

            
if len(indexDropList_mix) == 0:
    print("no mixtures found")
    
else:
    #drop mixtures
    df4 = df4.drop(df4.index[indexDropList_mix])
    
    print(f"{len(indexDropList_mix)} mixtures found")
    
    #save removes mixtures
    mixtures = pd.concat(mixtureList)
    mixtures.to_csv(f'{savepath}\\mixtures_after_2.csv', sep=',', header=True, index=False)
df4 

In [ ]:
#second std
"""
    -Standardize unknown stereochemistry (Handled by the RDKit Mol file parser)
        Fix wiggly bonds on sp3 carbons - sets atoms and bonds marked as unknown stereo to no stereo
        Fix wiggly bonds on double bonds – set double bond to crossed bond
    -Clears S Group data from the mol file
    -Kekulize the structure
    -Remove H atoms (See the page on explicit Hs for more details)
    -Normalization:
        Fix hypervalent nitro groups
        Fix KO to K+ O- and NaO to Na+ O- (Also add Li+ to this)
        Correct amides with N=COH
        Standardise sulphoxides to charge separated form
        Standardize diazonium N (atom :2 here: [*:1]-[N;X2:2]#[N;X1:3]>>[*:1]) to N+
        Ensure quaternary N is charged
        Ensure trivalent O ([*:1]=[O;X2;v3;+0:2]-[#6:3]) is charged
        Ensure trivalent S ([O:1]=[S;D2;+0:2]-[#6:3]) is charged
        Ensure halogen with no neighbors ([F,Cl,Br,I;X0;+0:1]) is charged
    -The molecule is neutralized, if possible. See the page on neutralization rules for more details.
    -Remove stereo from tartrate to simplify salt matching
    -Normalise (straighten) triple bonds and allenes
    
    https://github.com/chembl/ChEMBL_Structure_Pipeline
"""

rdMol = [Chem.MolFromSmiles(smile, sanitize=True) for smile in df4['thirdSaltRemoved_smiles']]

molBlock = [Chem.MolToMolBlock(mol) for mol in rdMol]

stdMolBlock = [standardizer.standardize_molblock(mol_block) for mol_block in molBlock]

molFromMolBlock = [Chem.MolFromMolBlock(std_molblock) for std_molblock in stdMolBlock]

mol2smiles = [Chem.MolToSmiles(m) for m in molFromMolBlock]

df4['Stand_smiles_final'] = mol2smiles